# Wprowadzanie tekstów piosenek

In [54]:
import pandas as pd
from dotenv import dotenv_values
from openai import OpenAI
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams
import uuid

In [ ]:
env = dotenv_values('.env')

EMBEDDING_MODEL = 'text-embedding-ada-002'
EMBEDDING_DIM = 1536
QDRANT_COLLECTION_NAME = 'dylans_songs'

openai_client = OpenAI(api_key=env['OPENAI_API_KEY'])


def get_qdrant_client():
    return QdrantClient(
        url=env["QDRANT_URL"], 
        api_key=env["QDRANT_API_KEY"],
    )

def assure_db_collection_exists():
    qdrant_client = get_qdrant_client()
    if not qdrant_client.collection_exists(QDRANT_COLLECTION_NAME):
        print('Tworzę kolekcję')
        qdrant_client.create_collection(
            collection_name=QDRANT_COLLECTION_NAME,
            vectors_config=VectorParams(
                size=EMBEDDING_DIM, 
                distance=Distance.COSINE,
            ),
        )
    else:
        print("Kolekcja już istnieje")

def get_embedding(text):
    result = openai_client.embeddings.create(
        input=[text],
        model=EMBEDDING_MODEL,
    )
    embedding = result.data[0].embedding
    tokens_used = result.usage.total_tokens # Pobranie liczby tokenów
    return embedding, tokens_used

def upload_song_to_qdrant(title, lyrics):
    qdrant_client = get_qdrant_client()
    embedding, tokens_used = get_embedding(lyrics)  # Generowanie embeddingu
    metadata = {
        "title": title,
        "lyrics": lyrics  # Tylko tekst piosenki w metadanych
    }
    # Dodanie pojedynczego wektora do kolekcji
    point_id = str(uuid.uuid4())
    point = {
        "id": point_id,  # Tytuł piosenki jako unikalny identyfikator
        "vector": embedding,  # Embedding piosenki
        "payload": metadata  # Metadane (tekst piosenki)
    }
    qdrant_client.upsert(
        collection_name=QDRANT_COLLECTION_NAME,
        points=[point]  # Tytuł jako klucz
    )
    print(f"Piosenka '{title}' została dodana do bazy.")
    return tokens_used

assure_db_collection_exists()

In [ ]:

while True:
    # Pobieranie danych od użytkownika
    title = input("Podaj tytuł piosenki (lub wpisz 'exit', aby zakończyć): ")
    if title.lower() == 'exit':
        break
    
    lyrics = input("Podaj tekst piosenki: ")
    
    # Dodawanie danych do DataFrame
    df = pd.concat([df, pd.DataFrame({"Title": [title], "Lyrics": [lyrics]})], ignore_index=True)
    print(f"Dodano: {title}")


In [ ]:
# Zmienna do śledzenia liczby użytych tokenów
total_tokens = 0

# Iteracja po DataFrame i wysyłanie danych pojedynczo
for _, row in df.iterrows():
    tokens_used = upload_song_to_qdrant(row['Title'], row['Lyrics'])
    total_tokens += tokens_used

# Wyświetlenie liczby użytych tokenów
print(f"Łączna liczba użytych tokenów: {total_tokens}")

In [ ]:

# Zapis DataFrame do pliku CSV
df.to_csv("dylans_songs.csv", index=False)
print("Piosenki zapisano w pliku 'dylans_songs.csv'.")
